# make sure the custom op work well

In [1]:
!conda info -e
import tensorflow as tf
import numpy as np
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
print(tf.__version__)

# conda environments:
#
base                     /home/gx/anaconda3
tf1.14                *  /home/gx/anaconda3/envs/tf1.14
tf2.0                    /home/gx/anaconda3/envs/tf2.0

1.14.1


/home/gx/anaconda3/envs/tf1.14/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gx/anaconda3/envs/tf1.14/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gx/anaconda3/envs/tf1.14/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/gx/anaconda3/envs/tf1.14/lib/py

In [48]:
n = 10
images = [[[[x * n + y + 1, x * n + y + 1, x * n + y + 1] for y in range(n)] for x in range(n)]]
print(images)
images = np.array(images,  dtype=np.int32)

[[[[1, 1, 1], [2, 2, 2], [3, 3, 3], [4, 4, 4], [5, 5, 5], [6, 6, 6], [7, 7, 7], [8, 8, 8], [9, 9, 9], [10, 10, 10]], [[11, 11, 11], [12, 12, 12], [13, 13, 13], [14, 14, 14], [15, 15, 15], [16, 16, 16], [17, 17, 17], [18, 18, 18], [19, 19, 19], [20, 20, 20]], [[21, 21, 21], [22, 22, 22], [23, 23, 23], [24, 24, 24], [25, 25, 25], [26, 26, 26], [27, 27, 27], [28, 28, 28], [29, 29, 29], [30, 30, 30]], [[31, 31, 31], [32, 32, 32], [33, 33, 33], [34, 34, 34], [35, 35, 35], [36, 36, 36], [37, 37, 37], [38, 38, 38], [39, 39, 39], [40, 40, 40]], [[41, 41, 41], [42, 42, 42], [43, 43, 43], [44, 44, 44], [45, 45, 45], [46, 46, 46], [47, 47, 47], [48, 48, 48], [49, 49, 49], [50, 50, 50]], [[51, 51, 51], [52, 52, 52], [53, 53, 53], [54, 54, 54], [55, 55, 55], [56, 56, 56], [57, 57, 57], [58, 58, 58], [59, 59, 59], [60, 60, 60]], [[61, 61, 61], [62, 62, 62], [63, 63, 63], [64, 64, 64], [65, 65, 65], [66, 66, 66], [67, 67, 67], [68, 68, 68], [69, 69, 69], [70, 70, 70]], [[71, 71, 71], [72, 72, 72], [7

In [49]:
images.shape

(1, 10, 10, 3)

In [78]:
def EIP(images,sizes,strides,rates,padding):
    farr = images.flatten()
    padding_row_num = (int) ((images.shape[1] + strides[1] - 1) / strides[1])
    padding_col_num = (int) ((images.shape[2] + strides[2] - 1) / strides[2])
    padding_cha_num = sizes[1]*sizes[2]*images.shape[3]

    res = []
    
    for i in range(padding_row_num):
        for j in range(padding_col_num):
            padding_anchor = j*strides[2] + i*strides[1]*images.shape[2]
            
            # pros one pad
            for m in range(sizes[1]):
                for n in range(sizes[2]): 
                    # pointer in this pad
                    pros_pointer = padding_anchor + n*rates[2] + m*rates[1]*images.shape[2]
                    pros_pointer *= images.shape[3]

                    isOutRow = (int)(padding_anchor / images.shape[2]) + m > images.shape[1]-1
                    isOutCol = padding_anchor % images.shape[2] + n > images.shape[2]-1

                    if isOutRow or isOutCol:
                        for ic in range(images.shape[3]):
                            res.append(0)
                        continue
                    # regular condition
                    for ic in range(images.shape[3]):
                        res.append(farr[pros_pointer+ic])
    return np.array(res).reshape(1,padding_row_num,padding_col_num,padding_cha_num)

In [79]:
print(images[0,:,:,0])
temp = EIP(images, sizes=[1, 4, 4, 1], strides=[1, 7, 7, 1], rates=[1, 1, 1, 1], padding='SAME')
print(temp)
print(temp.shape)

[[  1   2   3   4   5   6   7   8   9  10]
 [ 11  12  13  14  15  16  17  18  19  20]
 [ 21  22  23  24  25  26  27  28  29  30]
 [ 31  32  33  34  35  36  37  38  39  40]
 [ 41  42  43  44  45  46  47  48  49  50]
 [ 51  52  53  54  55  56  57  58  59  60]
 [ 61  62  63  64  65  66  67  68  69  70]
 [ 71  72  73  74  75  76  77  78  79  80]
 [ 81  82  83  84  85  86  87  88  89  90]
 [ 91  92  93  94  95  96  97  98  99 100]]
00 False
False
7.7
01 False
False
7.7
02 False
False
7.7
03 False
True
7.7
10 False
False
8.7
11 False
False
8.7
12 False
False
8.7
13 False
True
8.7
20 False
False
9.7
21 False
False
9.7
22 False
False
9.7
23 False
True
9.7
30 True
False
10.7
31 True
False
10.7
32 True
False
10.7
33 True
True
10.7
[[[[  1   1   1   2   2   2   3   3   3   4   4   4  11  11  11  12  12
     12  13  13  13  14  14  14  21  21  21  22  22  22  23  23  23  24
     24  24  31  31  31  32  32  32  33  33  33  34  34  34]
   [  8   8   8   9   9   9  10  10  10   0   0   0  18  18  18 

In [5]:
with tf.Session() as sess:
    res = tf.extract_image_patches(images=images, sizes=[1, 4, 4, 1], strides=[1, 7, 7, 1], rates=[1, 1, 1, 1], padding='SAME').eval()
    print(res)
    print(res.shape)

[[[[  1   1   1   2   2   2   3   3   3   4   4   4  11  11  11  12  12
     12  13  13  13  14  14  14  21  21  21  22  22  22  23  23  23  24
     24  24  31  31  31  32  32  32  33  33  33  34  34  34]
   [  8   8   8   9   9   9  10  10  10   0   0   0  18  18  18  19  19
     19  20  20  20   0   0   0  28  28  28  29  29  29  30  30  30   0
      0   0  38  38  38  39  39  39  40  40  40   0   0   0]]

  [[ 71  71  71  72  72  72  73  73  73  74  74  74  81  81  81  82  82
     82  83  83  83  84  84  84  91  91  91  92  92  92  93  93  93  94
     94  94   0   0   0   0   0   0   0   0   0   0   0   0]
   [ 78  78  78  79  79  79  80  80  80   0   0   0  88  88  88  89  89
     89  90  90  90   0   0   0  98  98  98  99  99  99 100 100 100   0
      0   0   0   0   0   0   0   0   0   0   0   0   0   0]]]]
(1, 2, 2, 48)


In [94]:
image = [j+i*85 for i in range(64) for j in range(85)]
image = np.array(image).reshape(1,64,85,1)
print(image[:,:,:,0])
print(image.shape)

[[[   0    1    2 ...   82   83   84]
  [  85   86   87 ...  167  168  169]
  [ 170  171  172 ...  252  253  254]
  ...
  [5185 5186 5187 ... 5267 5268 5269]
  [5270 5271 5272 ... 5352 5353 5354]
  [5355 5356 5357 ... 5437 5438 5439]]]
(1, 64, 85, 1)


In [89]:
with tf.Session() as sess:
    res = tf.extract_image_patches(image, sizes=[1, 3, 3, 1], strides=[1, 1, 1, 1], rates=[1, 1, 1, 1], padding='SAME')
    res = tf.reshape(res, [ -1, 3, 3, 1])
    res = tf.transpose(res, [1, 2, 3, 0])
    res = sess.run(res)
    print(res)
    print(res.shape)

[[[[   0    0    0 ... 5351 5352 5353]]

  [[   0    0    0 ... 5352 5353 5354]]

  [[   0    0    0 ... 5353 5354    0]]]


 [[[   0    0    1 ... 5436 5437 5438]]

  [[   0    1    2 ... 5437 5438 5439]]

  [[   1    2    3 ... 5438 5439    0]]]


 [[[   0   85   86 ...    0    0    0]]

  [[  85   86   87 ...    0    0    0]]

  [[  86   87   88 ...    0    0    0]]]]
(3, 3, 1, 5440)


In [3]:
interpreter = tf.lite.Interpreter(model_path='/home/gx/myproj/generative_inpainting-master/_1q.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
print(str(input_details))
output_details = interpreter.get_output_details()
print(str(output_details))


[{'name': 'input_image_holder', 'index': 1, 'shape': array([ 1, 10, 10,  3], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0)}]
[{'name': 'ExtractImagePatches_7', 'index': 0, 'shape': array([ 1,  2,  2, 48], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0)}]


In [4]:
input_shape = input_details[0]['shape']
print("input_shape:")
print(input_shape)
# input_data = np.array(10*np.random.random_sample(input_shape), dtype=np.int32)

input_data = images
# print(input_data)

input_shape:
[ 1 10 10  3]


In [5]:
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)
print(output_data.shape)

[[[[  1   1   1   2   2   2   3   3   3   4   4   4  11  11  11  12  12
     12  13  13  13  14  14  14  21  21  21  22  22  22  23  23  23  24
     24  24  31  31  31  32  32  32  33  33  33  34  34  34]
   [  8   8   8   9   9   9  10  10  10   0   0   0  18  18  18  19  19
     19  20  20  20   0   0   0  28  28  28  29  29  29  30  30  30   0
      0   0  38  38  38  39  39  39  40  40  40   0   0   0]]

  [[ 71  71  71  72  72  72  73  73  73  74  74  74  81  81  81  82  82
     82  83  83  83  84  84  84  91  91  91  92  92  92  93  93  93  94
     94  94   0   0   0   0   0   0   0   0   0   0   0   0]
   [ 78  78  78  79  79  79  80  80  80   0   0   0  88  88  88  89  89
     89  90  90  90   0   0   0  98  98  98  99  99  99 100 100 100   0
      0   0   0   0   0   0   0   0   0   0   0   0   0   0]]]]
(1, 2, 2, 48)


In [13]:
[[[[  1   1   1   2   2   2   3   3   3   4   4   4  11  11  11  12  12
     12  13  13  13  14  14  14  21  21  21  22  22  22  23  23  23  24
     24  24  31  31  31  32  32  32  33  33  33  34  34  34]
   [  8   8   8   9   9   9  10  10  10   0   0   0  18  18  18  19  19
     19  20  20  20   0   0   0  28  28  28  29  29  29  30  30  30   0
      0   0  38  38  38  39  39  39  40  40  40   0   0   0]]

  [[ 71  71  71  72  72  72  73  73  73  74  74  74  81  81  81  82  82
     82  83  83  83  84  84  84  91  91  91  92  92  92  93  93  93  94
     94  94   0   0   0   0   0   0   0   0   0   0   0   0]
   [ 78  78  78  79  79  79  80  80  80   0   0   0  88  88  88  89  89
     89  90  90  90   0   0   0  98  98  98  99  99  99 100 100 100   0
      0   0   0   0   0   0   0   0   0   0   0   0   0   0]]]]
(1, 2, 2, 48)

SyntaxError: invalid syntax (<ipython-input-13-2bde2cbd7211>, line 1)